In [1]:
from signals import loadData
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

In [6]:
import strategies
# test_data = loadData(287*180, False)[287*16:287*18]   #falling market
test_data = loadData(287*180, True)
sma = strategies.SMA(10, 50, is_statefull= False)
rsi = strategies.RSI(14, is_statefull= False)
macd = strategies.MACD(12, 26, 9, is_statefull= False)
composite = strategies.StateFullComposite(100000, cash_in_hand=100000, strategies=[sma, rsi, macd])
actions = []
amounts = []
for i in range(1, len(test_data)+1):
    action, amount = composite.action( test_data.head(i) )
    actions.append( action )
    amounts.append( amount )
test_data['signal'] = actions
test_data['amount'] = [ abs(amount) for amount in amounts ]

test_data['signal'].replace({0:'hold', 1: 'buy', -1:'sell', 2: 'strong_buy', 3:'very_strong_buy', -2: 'strong_sell', -3: 'very_strong_sell'}, inplace=True)
is_signal_col = lambda name: 'mid' in name or 'quote' in name or 'amount' in name or 'RSI' in name or 'MACD' in name or 'Signal' in name
line = px.line(test_data, x = 'timeStamp', y = [col for col in test_data.columns if is_signal_col(col)])
scatter = px.scatter(test_data[~test_data['signal'].isin(['hold'])], x = 'timeStamp', y='mid', color='signal', size = 'mid', hover_data=['amount'] )
plot = go.Figure(data=line.data + scatter.data)
plot.show()

bought 1903 units at 1.05262/unit
total holding: 1903 at avg buy price: 1.05262
cash available: 97996.86414 | total profit: 0
sold 1903 units at profit: 0.0018900000000001693 avg buy price: 1.05262 current price: 1.05451
total holding: 0 at avg buy price: 0
cash available: 100003.59667 | total profit: 3.596670000000322
bought 4342 units at 1.05533/unit
total holding: 4342 at avg buy price: 1.05533
cash available: 95421.35381 | total profit: 3.596670000000322
sold 2886 units at profit: 0.00012999999999996348 avg buy price: 1.05533 current price: 1.05546
total holding: 1456 at avg buy price: 1.0550723214285715
cash available: 98467.41137 | total profit: 3.9718500000002166
sold 1456 units at profit: 0.0030676785714285337 avg buy price: 1.0550723214285715 current price: 1.05814
total holding: 0 at avg buy price: 0
cash available: 100008.06321000001 | total profit: 8.438390000000162
bought 568 units at 1.05801/unit
total holding: 568 at avg buy price: 1.05801
cash available: 99407.11353 | t

In [3]:
test_data = loadData(287*180, False)[287*16:287*18]
is_signal_col = lambda name: 'mid' in name
line = px.line(test_data, x = 'timeStamp', y = [col for col in test_data.columns if is_signal_col(col)])
line.show()